In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
from scipy import signal
from sklearn.cross_decomposition import CCA
from scipy.signal import kaiserord, lfilter, firwin, freqz, butter, filtfilt, convolve
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

## Load Dataset

In [3]:
dataset_name = "recordings/psychopy_sultan_recording_" # 0 to 3
num_targets = 4
targets = [None] * num_targets
channels = ['P7', 'O1', 'O2', 'P8'] # data channels
for i in range(num_targets):
    targets[i] = pd.read_csv(dataset_name+str(i)+'.csv')
    targets[i]['Unnamed: 0'] = [i % 120 for i in range(len(targets[i].index))]
    targets[i].rename(columns = {'Unnamed: 0': 'Counter'}, inplace=True)
targets[0].head

<bound method NDFrame.head of      Counter  O1/theta  O1/alpha  O1/betaL  O1/betaH  O1/gamma  O2/theta  \
0          0     0.598     0.750     0.959     0.850     0.640     0.630   
1          1     0.725     0.918     1.052     0.883     0.582     0.568   
2          2     0.809     1.082     1.081     0.868     0.529     0.494   
3          3     0.808     1.178     1.037     0.801     0.489     0.415   
4          4     0.721     1.161     0.937     0.695     0.466     0.354   
..       ...       ...       ...       ...       ...       ...       ...   
595      115     0.299     0.428     0.171     0.174     0.184     0.091   
596      116     0.357     0.420     0.189     0.184     0.190     0.090   
597      117     0.406     0.452     0.198     0.197     0.196     0.095   
598      118     0.433     0.521     0.195     0.206     0.205     0.106   
599      119     0.427     0.611     0.186     0.215     0.217     0.124   

     O2/alpha  O2/betaL  O2/betaH  O2/gamma          TIME

In [4]:
def cut_uneven_datasets(df_list):
    # use this function if the datasets are of uneven length - chop to minimum df size
    cutoff_list = [len(df_list[i].index) for i in range(num_targets)]
    cutoff = min(cutoff_list)
    for i in range(len(cutoff_list)):
        keep_indicies = set(range(df_list[i].shape[0])) - set(range(cutoff, df_list[i].shape[0]))
        df_list[i] = df_list[i].take(list(keep_indicies))
        # make sure each df same length
        print('new length of dataset {} = {}'.format(i+1, len(df_list[i].index)))
    return df_list
        
# check size - make sure each target is the same length
for i in range(num_targets):
    print('length target {} = {}'.format(i, len(targets[i].index)))

# fix length
# targets = cut_uneven_datasets(targets)

length target 0 = 600
length target 1 = 600
length target 2 = 600
length target 3 = 600


In [4]:
# copy unfiltered data
original_data = [None] * num_targets
for i in range(num_targets):
    original_data[i] = pd.read_csv(dataset_name+str(i)+'.csv')
    original_data[i]['Unnamed: 0'] = [i % 120 for i in range(len(original_data[i].index))]
    original_data[i].rename(columns = {'Unnamed: 0': 'Counter'}, inplace=True)